In [28]:
import pandas as pd
import numpy as np
from nltk.stem.porter import PorterStemmer
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
df = pd.read_csv('disney_plus_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,A Spark Story,"Jason Sterman, Leanne Dare","Apthon Corbin, Louis Gonzales",NaN,"September 24, 2021",2021,TV-PG,88 min,Documentary,Two Pixar filmmakers strive to bring their uni...
1,s2,Movie,Spooky Buddies,Robert Vince,"Tucker Albrizzi, Diedrich Bader, Ameko Eks Mas...","United States, Canada","September 24, 2021",2011,G,93 min,"Comedy, Fantasy, Kids",The puppies go on a spooky adventure through a...
2,s3,Movie,The Fault in Our Stars,Josh Boone,"Shailene Woodley, Ansel Elgort, Laura Dern, Sa...",United States,"September 24, 2021",2014,PG-13,127 min,"Coming of Age, Drama, Romance",Hazel and Gus share a love that sweeps them on...
3,s4,TV Show,Dog: Impossible,NaN,Matt Beisner,United States,"September 22, 2021",2019,TV-PG,2 Seasons,"Animals & Nature, Docuseries, Family",Matt Beisner uses unique approaches to modifyi...
4,s5,TV Show,Spidey And His Amazing Friends,NaN,"Benjamin Valic, Lily Sanfelippo, Jakari Fraser...",United States,"September 22, 2021",2021,TV-Y,1 Season,"Action-Adventure, Animation, Kids",Spidey teams up with pals to become The Spidey...


In [3]:
df.isnull().sum()

show_id           0
type              0
title             0
director        440
cast            174
country         175
date_added        3
release_year      0
rating            2
duration          0
listed_in         0
description       0
dtype: int64

In [4]:
df = df.dropna()

In [5]:
movie_df = df[df['type'] == 'Movie']
movie_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1,s2,Movie,Spooky Buddies,Robert Vince,"Tucker Albrizzi, Diedrich Bader, Ameko Eks Mas...","United States, Canada","September 24, 2021",2011,G,93 min,"Comedy, Fantasy, Kids",The puppies go on a spooky adventure through a...
2,s3,Movie,The Fault in Our Stars,Josh Boone,"Shailene Woodley, Ansel Elgort, Laura Dern, Sa...",United States,"September 24, 2021",2014,PG-13,127 min,"Coming of Age, Drama, Romance",Hazel and Gus share a love that sweeps them on...
6,s7,Movie,Confessions of a Shopaholic,P.J. Hogan,"Isla Fisher, Hugh Dancy, Joan Cusack, John Goo...",United States,"September 17, 2021",2009,PG,106 min,"Comedy, Romance, Romantic Comedy",Becky writes a personal finance column en rout...
18,s19,Movie,Pirates of the Caribbean: On Stranger Tides,Rob Marshall,"Johnny Depp, Penélope Cruz, Ian McShane, Geoff...","United States, United Kingdom","September 10, 2021",2011,PG-13,140 min,"Action-Adventure, Fantasy",A woman from his past uses Jack to help find t...
19,s20,Movie,Twenty Something,Aphton Corbin,"Kaylin Price, Ariana Brown, Aliyah Taylor, Jan...",United States,"September 10, 2021",2021,PG,11 min,"Animation, Family","Adulting is hard. One day you're nailing it, t..."


In [6]:
movie_df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [7]:
movie_df = movie_df[['title', 'director', 'cast', 'release_year', 'rating', 'duration', 'listed_in', 'description']].reset_index()
movie_df.head()

,index,title,director,cast,release_year,rating,duration,listed_in,description
0,1,Spooky Buddies,Robert Vince,"Tucker Albrizzi, Diedrich Bader, Ameko Eks Mas...",2011,G,93 min,"Comedy, Fantasy, Kids",The puppies go on a spooky adventure through a...
1,2,The Fault in Our Stars,Josh Boone,"Shailene Woodley, Ansel Elgort, Laura Dern, Sa...",2014,PG-13,127 min,"Coming of Age, Drama, Romance",Hazel and Gus share a love that sweeps them on...
2,6,Confessions of a Shopaholic,P.J. Hogan,"Isla Fisher, Hugh Dancy, Joan Cusack, John Goo...",2009,PG,106 min,"Comedy, Romance, Romantic Comedy",Becky writes a personal finance column en rout...
3,18,Pirates of the Caribbean: On Stranger Tides,Rob Marshall,"Johnny Depp, Penélope Cruz, Ian McShane, Geoff...",2011,PG-13,140 min,"Action-Adventure, Fantasy",A woman from his past uses Jack to help find t...
4,19,Twenty Something,Aphton Corbin,"Kaylin Price, Ariana Brown, Aliyah Taylor, Jan...",2021,PG,11 min,"Animation, Family","Adulting is hard. One day you're nailing it, t..."


In [8]:
movie_df = movie_df.drop('index', axis=1)

In [9]:
movie_df['tags'] = movie_df['director'] + movie_df['cast'] + str(movie_df['release_year']) + movie_df['rating'] + movie_df['duration'] + movie_df['listed_in'] + movie_df['description']
movie_df = movie_df.drop(['director', 'cast', 'release_year', 'rating', 'duration', 'listed_in', 'description'], axis=1)
movie_df.head()

,title,tags
0,Spooky Buddies,"Robert VinceTucker Albrizzi, Diedrich Bader, A..."
1,The Fault in Our Stars,"Josh BooneShailene Woodley, Ansel Elgort, Laur..."
2,Confessions of a Shopaholic,"P.J. HoganIsla Fisher, Hugh Dancy, Joan Cusack..."
3,Pirates of the Caribbean: On Stranger Tides,"Rob MarshallJohnny Depp, Penélope Cruz, Ian Mc..."
4,Twenty Something,"Aphton CorbinKaylin Price, Ariana Brown, Aliya..."


In [10]:
ps = PorterStemmer()
for i in range(0, len(movie_df['tags'])):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', movie_df['tags'][i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [11]:
movie_df.shape

(788, 2)

In [20]:
cv = CountVectorizer(max_features=800, lowercase=True)
movie_vector = cv.fit_transform(movie_df['tags'])

In [21]:
movie_vector.shape

(788, 800)

In [23]:
similarity = cosine_similarity(movie_vector)
similarity

array([[1.        , 0.62184878, 0.60526316, ..., 0.62126074, 0.62184878,
        0.55068879],
       [0.62184878, 1.        , 0.59481188, ..., 0.63828474, 0.63888889,
        0.59149617],
       [0.60526316, 0.59481188, 1.        , ..., 0.67773899, 0.62184878,
        0.5757201 ],
       ...,
       [0.62126074, 0.63828474, 0.67773899, ..., 1.        , 0.63828474,
        0.61779761],
       [0.62184878, 0.63888889, 0.62184878, ..., 0.63828474, 1.        ,
        0.59149617],
       [0.55068879, 0.59149617, 0.5757201 , ..., 0.61779761, 0.59149617,
        1.        ]])

In [33]:
def recommend(movie):
    movie_index = movie_df[movie_df['title'] == movie].index[0]
    distances = similarity [movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:11]
    
    for i in movie_list:
        print(movie_df.iloc[i[0]].title)
        
recommend('Before the Flood')

Drain The Great Lakes
Stonehenge Decoded: Secrets Revealed
Black Is King
Surviving the Mount St. Helens Disaster
Oklahoma!
Pirates of the Caribbean: Dead Men Tell No Tales
Cars Toon: Mater Private Eye
Petra: Secrets of the Ancient Builders
Drain The Sunken Pirate City
Mission to the Sun


In [30]:
pickle.dump('df.pkl', open('movie_df','wb'))

In [31]:
pickle.dump('similarity.pkl', open('similarity','wb'))